<a href="https://colab.research.google.com/github/GN-Yu/TSRL-project/blob/main/one_dimension.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import psutil
import tensorflow as tf
import subprocess

# Function to execute shell command and return output
def run_shell_command(cmd):
    return subprocess.run(cmd, stdout=subprocess.PIPE, shell=True).stdout.decode('utf-8').strip()

# Check CPU
cpu_info = run_shell_command("cat /proc/cpuinfo | grep 'model name' | uniq")
print("CPU:", cpu_info.split(":")[1].strip() if cpu_info else "Not available")

# Number of CPU Cores
num_cores = os.cpu_count()
print("Number of CPU cores:", num_cores)

# Total RAM
ram_info = psutil.virtual_memory()
total_ram = ram_info.total / (1024 ** 3)  # Convert bytes to GB
print(f"Total RAM : {total_ram} GB")

# Check GPU
gpu_info = !nvidia-smi --query-gpu=gpu_name,memory.total --format=csv
if gpu_info and len(gpu_info) > 1:
    gpu_name = gpu_info[1].split(',')[0]
    gpu_memory_mib = float(gpu_info[1].split()[2])  # Extract the memory in MiB
    gpu_memory_gb = gpu_memory_mib / 1024
    print(f"GPU: {gpu_name}, {gpu_memory_gb} GB")
else:
    print("GPU: Not available")

# TensorFlow info
print(f"TensorFlow version: {tf.__version__}")
print("TensorFlow GPU Availability:", tf.test.is_gpu_available())

CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
Number of CPU cores: 2
Total RAM : 12.674789428710938 GB


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


GPU: Tesla T4, 15.0 GB
TensorFlow version: 2.14.0
TensorFlow GPU Availability: True


In [2]:

# organize files
import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/TSRL_project'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [3]:
import numpy as np

# Set random seed
SEED = 2023
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [4]:
def sample_brownian_motion(num_trajectories, time_steps):
    """
    Function to sample Brownian motion for the specified number of trajectories and time steps.

    :param num_trajectories: The number of trajectories to simulate.
    :param time_steps: A numpy array of time steps.
    :return: A numpy array of shape (num_trajectories, len(time_steps)-1) representing the Brownian increments.
    """
    # Calculate time intervals (delta_t) as differences between consecutive time steps
    delta_t = np.diff(time_steps)

    # Sample Brownian increments ∆W for each time interval and each trajectory
    brownian_increments = np.random.normal(0, np.sqrt(delta_t), (num_trajectories, len(delta_t)))

    return brownian_increments

def sample_initial_states(num_samples):
    # Implement initial state sampling here
    pass



In [15]:
def sample_levy_jumps(num_trajectories, lambda_param, phi_function, time_horizon):
    """
    Function to sample jumps from a Lévy process for the specified number of trajectories.

    :param num_trajectories: The number of trajectories to simulate.
    :param lambda_param: The intensity parameter λ of the Poisson process.
    :param phi_function: The density function ϕ(z) of the jump sizes.
    :param time_horizon: The total time horizon T.
    :return: A list containing tuples (jump_times, jump_sizes) for each trajectory.
    """
    jumps = []

    for _ in range(num_trajectories):
        # Step (a): Generate a sequence of exponential distributions with parameter λ
        exponential_samples = np.random.exponential(1/lambda_param, 1000)  # Generate a large number of samples

        # Step (b): Compute the cumulative sum to get the arrival times of the Poisson process
        arrival_times = np.cumsum(exponential_samples)
        arrival_times = arrival_times[arrival_times <= time_horizon]  # Filter times beyond the time horizon

        # Step (c): Sample from ϕ(z) for each arrival time to get the jump sizes
        jump_sizes = np.array([np.random.normal(0, 1) for _ in arrival_times])

        jumps.append((arrival_times, jump_sizes))

    return jumps

# Example usage
lambda_param = 0.3  # Intensity parameter λ of the Poisson process
time_horizon = 1    # Total time horizon T
M = 1000  # Number of samples/trajectories

# Example density function ϕ(z) (e.g., normal distribution)
def phi_function():
    return np.random.normal(0, 1)  # Replace with the actual density function used in your problem

np.random.seed(SEED)

# Simulate jumps for a specific number of trajectories
levy_jumps = sample_levy_jumps(M, lambda_param, phi_function, time_horizon)

# Example output of the first trajectory's jumps
for i in range(10):
  print(levy_jumps[i])  # (array of arrival times, array of jump sizes)


print(len(levy_jumps))

(array([], dtype=float64), array([], dtype=float64))
(array([], dtype=float64), array([], dtype=float64))
(array([0.83590594]), array([1.51761877]))
(array([0.082762]), array([-0.11873037]))
(array([], dtype=float64), array([], dtype=float64))
(array([0.62719477]), array([-1.14039301]))
(array([], dtype=float64), array([], dtype=float64))
(array([], dtype=float64), array([], dtype=float64))
(array([0.80695252]), array([-0.46033369]))
(array([0.44614681, 0.99265044]), array([ 0.15199075, -0.01471334]))
1000


In [ ]:
def create_neural_network(input_dim, layer_width=25, output_dim=2):
    # Define the input layer
    inputs = tf.keras.Input(shape=(input_dim,))

    # First linear layer
    x = tf.keras.layers.Dense(layer_width, activation='tanh')(inputs)

    # Add five residual blocks
    for _ in range(5):
        y = tf.keras.layers.Dense(layer_width, activation='tanh')(x)
        y = tf.keras.layers.Dense(layer_width, activation='tanh')(y)
        x = tf.keras.layers.Add()([x, y])

    # Final linear layer
    outputs = tf.keras.layers.Dense(output_dim)(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
d = 1
input_dim = d + 1  # Set 'd' as problem's dimensionality
model = create_neural_network(input_dim)

In [25]:
mu = 0.4
sigma = 0.25
z = 0.5
phi = np.random.normal(mu, sigma, z)

TypeError: ignored

In [23]:
# Define Constants

lambda_poisson = 0.3
mu = 0.4
sigma = 0.25

def phi_function():
    return np.random.normal(0, 1)

def G_function(x, z):
  return x * np.exp(z)

T = 1.0  # Total time
N = 50  # Number of time intervals
M = 1000  # Number of samples/trajectories
Iterations = 400  # Number of iterations

initial_learning_rate = 5e-5

# Generate equally spaced time snapshots from 0 to T
time_steps = np.linspace(0, T, N+1)


for iteration in range(Iterations):
  brownian_motions = sample_brownian_motion(M, time_steps)
  levy_jumps = sample_levy_jumps(M, lambda_poisson, phi_function, T)

  X = [[0 for _ in range(M)] for _ in range(N+1)]
  Y = [[0 for _ in range(M)] for _ in range(N+1)]

  for timestep, time in enumerate(time_steps):
    if timestep == 0:
      X[timestep] = [1 for _ in range(M)]  # Initial value for PIDE
    else:
      time_pre = time_steps[timestep-1]
      time_diff = time - time_pre

      for trajectory in range(M):
        arrival_times, jump_sizes = levy_jumps[trajectory]
        mask = (arrival_times > time_pre) & (arrival_times <= time)

        X[timestep] = X[timestep] + G_function(X[timestep][trajectory], jump_sizes[mask]).sum() + time_diff *

KeyboardInterrupt: ignored